In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#import seaborn as sns
#%matplotlib inline
import re
import time
from datetime import datetime
import matplotlib.dates as mdates
import matplotlib.ticker as ticker
from urllib.request import urlopen
from bs4 import BeautifulSoup
import requests

In [49]:
#Quantidade maxima de páginas a serem extraidas
no_pages = 2

#Termo de busca digitado pelo user
search = "dan+brown"

def get_data(pageNo, searchTitle):
    #Configuracao do BeatifulSoup
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    # MODELO DE URL DO SITE - https://www.amazon.com.br/s?k=dan+brown&i=stripbooks&page=2&__mk_pt_BR=?ref=sr_pg_2
    r = requests.get('https://www.amazon.com.br/s?k='+str(searchTitle)+'&i=stripbooks&page='+str(pageNo)+'&__mk_pt_BR=?ref=sr_pg_'+str(pageNo), headers=headers)#, proxies=proxies)
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup)

    alls = []
    
    #Encontra no html todas as divs com essa class e pega seu conteúdo
    for d in soup.findAll('div', attrs={'class':'s-include-content-margin s-latency-cf-section s-border-bottom s-border-top'}):
        #print(d)
        name = d.find('span', attrs={'class':'a-size-medium a-color-base a-text-normal'})
        #n = name.find_all('img', alt=True)
        #print(n[0]['alt'])
        author = d.find_all('span', attrs={'class':'a-size-base'})
        
        #print(len(author))
        #print(author[3])
        if len(author) >= 6:
            author = author[3] 
        else:
            author = author[1]

        #rating = d.find('span', attrs={'class':'4,1 de 5 estrelas'})
        #r = rating.find_all('span', aria-label=True)
        
        price = d.find('span', attrs={'class':'a-offscreen'})

        imgUrl = d.find('img', attrs={'class': 's-image'})['src']
        
        siteUrl = "www.amazon.com.br"
        siteUrl += d.find('a', attrs={'class': 'a-link-normal s-no-outline'})['href']
        
        all1=[]

        if name is not None:
            #print(n[0]['alt'])
            all1.append(name.text)
        else:
            all1.append("unknown-product")

        if author is not None:
            #print(author.text)
            all1.append(author.text)
        elif author is None:
            author = d.find('span', attrs={'class':'a-size-small a-color-base'})
            if author is not None:
                all1.append(author.text)
            else:    
                all1.append('0')
        
        if imgUrl is not None:
            all1.append(imgUrl)
        else:
            all1.append("invalid-image")
        
        if siteUrl is not None:
            all1.append(siteUrl)
        else:
            all1.append("invalid-url-site")

        #if rating is not None:
            #print(rating.text)
        #    all1.append(rating.text)
        #else:
        #    all1.append('-1')

        #if users_rated is not None:
            #print(price.text)
        #    all1.append(users_rated.text)
        #else:
        #    all1.append('0')     

        if price is not None:
            #print(price.text)
            all1.append(price.text)
        else:
            all1.append('0')
        alls.append(all1)    
    return alls

In [50]:
results = []
for i in range(1, no_pages+1):
    results.append(get_data(i, search))
flatten = lambda l: [item for sublist in l for item in sublist]
df = pd.DataFrame(flatten(results),columns=['Titulo','Autor', 'ImgUrl', 'SiteUrl', 'Preco'])
df.to_csv('amazon_products.csv', index=False, encoding='utf-8')

In [51]:
df = pd.read_csv("amazon_products.csv")
df.shape

(44, 5)

In [52]:
df.head(61)

,Titulo,Autor,ImgUrl,SiteUrl,Preco
0,O código Da Vinci,Dan Brown,https://m.media-amazon.com/images/I/710DjekSUk...,www.amazon.com.br/gp/slredirect/picassoRedirec...,"R$ 19,95"
1,O terceiro gêmeo,Ken Follett,https://m.media-amazon.com/images/I/81HGFYD5iR...,www.amazon.com.br/gp/slredirect/picassoRedirec...,"R$ 36,60"
2,Origem (Robert Langdon),Dan Brown,https://m.media-amazon.com/images/I/81Y3DwapTV...,www.amazon.com.br/Origem-Dan-Brown/dp/85804176...,"R$ 43,90"
3,O símbolo perdido,Dan Brown,https://m.media-amazon.com/images/I/91yUh4+m2g...,www.amazon.com.br/s%C3%ADmbolo-perdido-Dan-Bro...,"R$ 19,90"
4,Ponto de Impacto,Dan Brown,https://m.media-amazon.com/images/I/716SIRfELk...,www.amazon.com.br/Ponto-Impacto-Dan-Brown/dp/8...,"R$ 36,10"
5,Anjos e demônios (Robert Langdon),Dan Brown,https://m.media-amazon.com/images/I/91mqo8pial...,www.amazon.com.br/Anjos-dem%C3%B4nios-Dan-Brow...,"R$ 35,91"
6,Sinfonia dos animais,Dan Brown,https://m.media-amazon.com/images/I/91opv6Zanw...,www.amazon.com.br/Sinfonia-dos-animais-Dan-Bro...,"R$ 29,00"
7,O código Da Vinci,Dan Brown,https://m.media-amazon.com/images/I/710DjekSUk...,www.amazon.com.br/c%C3%B3digo-Vinci-Dan-Brown/...,"R$ 19,95"
8,Inferno (Robert Langdon),Dan Brown,https://m.media-amazon.com/images/I/71NoHFvBj-...,www.amazon.com.br/Inferno-Dan-Brown/dp/8580416...,"R$ 50,00"
9,Inferno: Featuring Robert Langdon: 4,Dan Brown,https://m.media-amazon.com/images/I/A1yLZ8yNC2...,www.amazon.com.br/Inferno-Dan-Brown/dp/0385537...,"R$ 74,99"
